## 3. Database

### Variante mit Umwandlung von data

In [ ]:
import psycopg2

# Datenbankverbindung herstellen
conn = psycopg2.connect("host=db dbname=postgres user=admin password=secret")

# Cursor-Objekt erstellen
cur = conn.cursor()

# Tabelle erstellen
cur.execute('''
    CREATE TABLE aktienkurse (
        index SERIAL PRIMARY KEY,
        datum DATE,
        open DECIMAL,
        high DECIMAL,
        low DECIMAL,
        close DECIMAL,
        adj_close DECIMAL,
        volume INTEGER
    );
''')

# Daten in die Tabelle einfügen
for row in data.itertuples(index=False):
    cur.execute('''
        INSERT INTO aktienkurse (datum, open, high, low, close, adj_close, volume)
        VALUES (%s, %s, %s, %s, %s, %s, %s);
    ''', row)

# Neue Spalte 'positive' zur Tabelle 'aktienkurse' hinzufügen
cur.execute("ALTER TABLE aktienkurse ADD COLUMN positive INTEGER DEFAULT 0;")

# Änderungen speichern
conn.commit()

# Datenbankverbindung schließen
cur.close()
conn.close()

cur.execute("UPDATE aktienkurse SET positive = CASE WHEN close >= open THEN 1 ELSE 0 END;")

# Änderungen speichern
conn.commit()

# Datenbankverbindung schließen
cur.close()
conn.close()

### Variante direkt mit data

In [ ]:
# Libraries
import os
import fnmatch
import tempfile
import psycopg2
import pandas as pd
from sqlalchemy import create_engine

os.environ['MPLCONFIGDIR'] = "/home/jovyan"
import matplotlib.pyplot as plt

# Settings
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Connect DB
conn = psycopg2.connect("host=db dbname=postgres user=admin password=secret")

In [ ]:
# Insert data to appl_prices
engine = create_engine('postgresql://admin:secret@db:5432/postgres')
data.to_sql('appl_prices', engine, if_exists='replace')
cur = conn.cursor()

# Spalte positive hinzufügen
cur.execute("ALTER TABLE appl_prices ADD COLUMN Positive INTEGER DEFAULT 0;")

# Änderungen speichern
conn.commit()

# Datenbankverbindung schließen
cur.close()
conn.close()

cur.execute("""UPDATE appl_prices SET positive = CASE WHEN "Close" >= "Open" THEN 1 ELSE 0 END;""")
